In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import plotly.express as px

In [ ]:
df_patientele = pd.read_csv("Bases de données/Données_Patientele_Departementale.csv", sep=";")
df_patientele['nombre_patients_uniques'] = pd.to_numeric(df_patientele['nombre_patients_uniques'],errors='coerce')
df_patientele.columns = df_patientele.columns.str.replace("\ufeff", "", regex=False).str.strip()

df_patientele_2017 = df_patientele[(df_patientele['annee'] == 2017) & (df_patientele['profession_sante'] == "Médecins généralistes (hors médecins à expertise particulière - MEP)")]           

In [ ]:
df_patientele.head()

## Carte des pays différents

In [ ]:
# Téléchargement des informations géospatiales des départements français (DROM compris)

departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# On modifie le code des départements : on rajoute un 0 à ceux qui ont un code à deux chiffres
# departements['INSEE_DEP'] = departements['INSEE_DEP'].str.zfill(3)

# Test d'affichage de la carte
departements.plot().axis('off')

In [ ]:
Carte_2017 = df_patientele_2017.copy()
Carte_2017['INSEE_DEP'] = Carte_2017['departement'].str[:3].astype(str)
Carte_2017 = departements.merge(Carte_2017, on='INSEE_DEP', how='left')

In [ ]:
# Tracé pour l'année 2017

vmin = Carte_2017['nombre_patients_uniques'].quantile(0.05)
vmax = Carte_2017['nombre_patients_uniques'].quantile(0.95)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
Carte_2017.plot(
    column='nombre_patients_uniques', 
    cmap='OrRd',
    vmin=vmin,
    vmax=vmax,      
    linewidth=0.4,
    edgecolor='black',
    legend=True,       
    ax=ax
)

# On supprime les axes
ax.axis('off')  
ax.set_title("Densité par département", fontsize=15)
plt.show()

## Graphique Evolution par année

In [ ]:
df_medecin = df_patientele[(df_patientele['profession_sante'] == "Médecins généralistes (hors médecins à expertise particulière - MEP)")]  
df_graphique_évol = df_medecin[["annee","libelle_departement","patients_uniques_integer"]]
df_graphique_évol.info

In [ ]:
L = []

df = df_graphique_évol[df_graphique_évol["libelle_departement"] == "Paris"]
df['annee'] = pd.to_numeric(df['annee'],errors='coerce')
df = df.sort_values(by='annee')
df.head()
plt.plot(df['annee'],
        df['patients_uniques_integer'],
        alpha=0.7
    )

plt.xlabel("Année")
plt.ylabel("Nombre de patients uniques")
plt.title("Évolution du nombre de patients uniques par région")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


df.head(10)

In [ ]:
df_graphique_évol = df_graphique_évol.sort_values(by='annee')

for region in df_graphique_évol['libelle_departement'].unique():
    tmp = df_graphique_évol[df_graphique_évol['libelle_departement'] == region]
    plt.plot(
        tmp['annee'],
        tmp['patients_uniques_integer'],
        label=region,
        alpha=0.7
    )

plt.xlabel("Année")
plt.ylabel("Nombre de patients uniques")
plt.title("Évolution du nombre de patients uniques par région")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
df.head(10)
df.shape
df['annee'].value_counts().sort_index()

In [ ]:
df = pd.read_excel("Bases de données/medecins_commune_2025.xlsx")
df.head(10)

In [ ]:
# code commune en string à 5 chiffres
df["Cartographie Interactive de la Démographie Médicale"] = df["Cartographie Interactive de la Démographie Médicale"].astype(str).str.zfill(5)

df.head()

In [ ]:
gdf = carti_download(
    values=df.set_index("Cartographie Interactive de la Démographie Médicale")["Unnamed: 2"],
    level="COMMUNE",
    year=2023,
    simplify=True
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

gdf.plot(
    column="nb_medecins",
    ax=ax,
    legend=True,
    cmap="OrRd",
    missing_kwds={
        "color": "lightgrey",
        "label": "Données manquantes"
    }
)

ax.set_title("Nombre de médecins par commune", fontsize=14)
ax.axis("off")

plt.show()


In [ ]:
df_medecins_lieu_formation_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=5)
df_medecins_lieu_formation_complet.head()

df_medecins_lieu_formation= df_medecins_lieu_formation_complet[(df_medecins_lieu_formation_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_lieu_formation_complet['region'] == '00-Ensemble')
    & (df_medecins_lieu_formation_complet['specialites'] == '01-Médecine générale')
    ]
df_medecins_lieu_formation.head()
if (df_medecins_lieu_formation[['specialites', 'region', 'specialites_agregees']].nunique().eq(1).all()):
    df_medecins_lieu_formation.drop(columns=['specialites', 'region', 'specialites_agregees'],inplace=True)

df_medecins_lieu_formation.head()

In [ ]:
cols_effectifs = [c for c in df_medecins_lieu_formation.columns if c.startswith("effectif_")]
df_deltas = df_medecins_lieu_formation[cols_effectifs].diff(axis=1)
df_deltas.columns = [f"delta_{c.split('_')[1]}" for c in cols_effectifs]
df_deltas = df_deltas.drop(columns="delta_2012")
df = pd.concat([df_medecins_lieu_formation, df_deltas], axis=1)
df = df[df["region_diplome"] != "00-Ensemble"]
df.head()

In [ ]:
import matplotlib.pyplot as plt

delta_cols = [c for c in df.columns if c.startswith("delta_")]
years = [int(c.split("_")[1]) for c in delta_cols]

plt.figure(figsize=(10, 6))

for _, row in df.iterrows():
    plt.plot(
        years,
        row[delta_cols],
        marker="o",
        label=row["region_diplome"]
    )

plt.axhline(0, color="black", linestyle="--", linewidth=1)
plt.xlabel("Année")
plt.ylabel("Variation annuelle des effectifs")
plt.title("Évolution annuelle des effectifs de médecins généralistes")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
all_deltas = df[delta_cols].values.flatten()

plt.figure(figsize=(7, 5))
plt.hist(all_deltas, bins=20)
plt.axvline(0, color="black", linestyle="--")
plt.xlabel("Delta annuel")
plt.ylabel("Fréquence")
plt.title("Distribution des variations annuelles des effectifs")
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))

sns.heatmap(
    df.set_index("region_diplome")[delta_cols],
    center=0,
    cmap="RdBu_r",
    annot=False
)

plt.title("Heatmap des variations annuelles des effectifs")
plt.xlabel("Année")
plt.ylabel("")
plt.tight_layout()
plt.show()


In [ ]:
df["delta_total"] = df["effectif_2025"] - df["effectif_2012"]

In [ ]:
df_medecins_effectif_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=1)
df_medecins_age_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=2)
df_medecins_densite_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=3)

In [ ]:
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']!='0-Ensemble') 
    & (df_medecins_effectif['tranche_age']=='00-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")] 

In [ ]:
df_medecins_secteur = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=6)
df_medecins_secteur.head()

df_medecins_secteur_réduit= df_medecins_secteur[(df_medecins_secteur['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_secteur['region'] == '00-Ensemble')
    & (df_medecins_secteur['departement'] == '000-Ensemble')
    & (df_medecins_secteur['mode_exercice'] == '0-Ensemble')
    & (df_medecins_secteur['territoire'] == '0-France entière')
    & (df_medecins_secteur['specialites'] == '01-Médecine générale')
    ]
df_medecins_secteur_réduit.head()


In [ ]:
df_medecins_secteur_réduit = df_medecins_secteur_réduit[df_medecins_secteur_réduit["secteur_activite"] != "00-Ensemble"]

def plot_pie_secteur(df, annee, ax):
    df_annuel = df[['secteur_activite', f'activités_{annee}']]

    labels = df_annuel['secteur_activite'].unique()
    sizes = df_annuel[f'activités_{annee}'].unique()

    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    ax.set_title(f"{annee}")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, annee in zip(axes, [2012, 2019, 2025]):
    plot_pie_secteur(df_medecins_secteur_réduit, annee, ax)

fig.suptitle("Répartition des médecins par secteur d’activité", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
df_hopital = df_medecins_secteur_réduit[df_medecins_secteur_réduit["secteur_activite"] == "01-Hôpital public"]

# Colonnes d'activité
cols_activite = [c for c in df_hopital.columns if c.startswith("activités_")]

# Années
annees = [int(c.split("_")[1]) for c in cols_activite]


plt.figure(figsize=(8, 5))
plt.plot(annees, df_hopital[cols_activite].iloc[0], marker='o')

plt.xlabel("Année")
plt.ylabel("Activité hospitalière")
plt.title("Évolution de l’activité hospitalière\nMédecins généralistes – France entière")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
df_medecins_effectif_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=1)

In [ ]:
# Traitement de la feuille des effectifs : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_effectif = df_medecins_effectif_complet[(df_medecins_effectif_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_effectif_complet['sexe'] == '0-Ensemble')
    & (df_medecins_effectif_complet['exercice']=='0-Ensemble') 
    & (df_medecins_effectif_complet['region'] == '00-Ensemble') 
    & (df_medecins_effectif_complet['territoire'] == "0-France entière")
    & (df_medecins_effectif_complet['tranche_age'] != "00-Ensemble")
    & (df_medecins_effectif_complet['specialites'] == '00-Ensemble')
    ]

df_medecins_tranches = df_medecins_effectif.drop(axis = 1, columns = ['region','sexe', 'specialites', 'specialites_agregees','territoire','departement','exercice'], inplace = True)
df_medecins_effectif.reset_index(drop = True, inplace = True)

df_medecins_effectif

In [ ]:
def nouvelle_tranche(tranche):
    if tranche in ["01-moins de 30 ans", "02-entre 30 et 34 ans"]:
        return "Moins de 35 ans"
    elif tranche in ["03-entre 35 et 39 ans", "04-entre 40 et 44 ans"]:
        return "35–44 ans"
    elif tranche in ["05-entre 45 et 49 ans", "06-entre 50 et 54 ans"]:
        return "45–54 ans"
    elif tranche in ["07-entre 55 et 59 ans", "08-entre 60 et 64 ans"]:
        return "55–64 ans"
    else:
        return "65 ans et +"

df_medecins_effectif["nouvelle_tranche_age"] = df_medecins_effectif["tranche_age"].apply(nouvelle_tranche)
cols_effectifs = [c for c in df_medecins_effectif.columns if c.startswith("effectif_")]

df_medecins_nouvelles_tranches = df_medecins_effectif.groupby("nouvelle_tranche_age")[cols_effectifs].sum().reset_index()
df_medecins_nouvelles_tranches.head()



In [ ]:
df_medecins_nouvelles_tranches.columns = ["Nouvelle_tranche_age"] + [f"{a}" for a in range(2012, 2026)]

# 1. Sélection des années voulues (une sur deux, de 2013 à 2025)
annees = list(range(2013, 2026, 2))
cols = [f"{a}" for a in annees]

# 2. Mise en forme : tranches d’âge en colonnes, années en lignes
df_plot = df_medecins_nouvelles_tranches.set_index("Nouvelle_tranche_age")[cols].T

# 3. Passage en proportions (chaque année = 100 %)
#df_plot = df_plot.div(df_plot.sum(axis=1), axis=0)

# 4. Graphique en barres empilées
df_plot.plot(kind="bar",stacked=True,figsize=(10, 6))

plt.ylabel("Proportion")
plt.xlabel("Année")
plt.title("Répartition des médecins (Médecine Générale) par tranche d’âge\n(en proportion, une année sur deux)")
plt.legend(title="Tranche d’âge", bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()


In [ ]:
df_medecins_specialité = df_medecins_effectif_complet[(df_medecins_effectif_complet['sexe'] == '0-Ensemble')
    & (df_medecins_effectif_complet['departement'] == '000-Ensemble')
    & (df_medecins_effectif_complet['exercice']=='0-Ensemble') 
    & (df_medecins_effectif_complet['region'] == '00-Ensemble') 
    & (df_medecins_effectif_complet['territoire'] == "0-France entière")
    & (df_medecins_effectif_complet['tranche_age'] == "00-Ensemble")
    & (df_medecins_effectif_complet['specialites'] != '00-Ensemble')
    ]

df_medecins_specialité.drop(axis = 1, columns = ['region','specialites_agregees','sexe', 'tranche_age','territoire','departement','exercice'], inplace = True)
df_medecins_specialité.reset_index(drop = True, inplace = True)
df_medecins_specialité.head()

In [ ]:
def plot_pie_secteur(df, annee, ax):
    df_annuel = df[['specialites', f'effectif_{annee}']]

    labels = df_annuel['specialites'].unique()
    sizes = df_annuel[f'effectif_{annee}'].unique()

    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    ax.set_title(f"{annee}")

fig, axes = plt.subplots(3, 1, figsize=(20, 20))

for ax, annee in zip(axes, [2012, 2019, 2025]):
    plot_pie_secteur(df_medecins_specialité, annee, ax)

fig.suptitle("Répartition des médecins par secteur d’activité", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
df_medecins_parité = df_medecins_effectif_complet[(df_medecins_effectif_complet['sexe'] != '0-Ensemble')
    & (df_medecins_effectif_complet['departement'] == '000-Ensemble')
    & (df_medecins_effectif_complet['exercice']=='0-Ensemble') 
    & (df_medecins_effectif_complet['region'] == '00-Ensemble') 
    & (df_medecins_effectif_complet['territoire'] == "0-France entière")
    & (df_medecins_effectif_complet['tranche_age'] == "00-Ensemble")
    & (df_medecins_effectif_complet['specialites'] == '00-Ensemble')
    & (df_medecins_effectif_complet['specialites_agregees'] == '1-Médecine générale')
    ]

df_medecins_parité.drop(axis = 1, columns = ['region','specialites_agregees','specialites', 'tranche_age','territoire','departement','exercice'], inplace = True)
df_medecins_parité.reset_index(drop = True, inplace = True)
df_medecins_parité.head()

In [ ]:
def plot_pie_secteur(df, annee, ax):
    df_annuel = df[['sexe', f'effectif_{annee}']]

    labels = df_annuel['sexe'].unique()
    sizes = df_annuel[f'effectif_{annee}'].unique()

    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    ax.set_title(f"{annee}")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for ax, annee in zip(axes, [2012, 2025]):
    plot_pie_secteur(df_medecins_parité, annee, ax)

fig.suptitle("Parité Homme/Femme chez les médecins généralistes entre 2012 et 2025", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
df_medecins_parité = df_medecins_effectif_complet[(df_medecins_effectif_complet['sexe'] != '0-Ensemble')
    & (df_medecins_effectif_complet['departement'] == '000-Ensemble')
    & (df_medecins_effectif_complet['exercice']=='0-Ensemble') 
    & (df_medecins_effectif_complet['region'] == '00-Ensemble') 
    & (df_medecins_effectif_complet['territoire'] == "0-France entière")
    & (df_medecins_effectif_complet['tranche_age'] == "00-Ensemble")
    & (df_medecins_effectif_complet['specialites'] != '00-Ensemble')
    & (df_medecins_effectif_complet['specialites_agregees'] == '2-Autres spécialités')
    ]

df_medecins_parité.drop(axis = 1, columns = ['region','specialites_agregees','specialites', 'tranche_age','territoire','departement','exercice'], inplace = True)
df_medecins_parité.reset_index(drop = True, inplace = True)
df_medecins_parité.head()

In [ ]:
df_medecins_nouvelles_tranches.columns = ["Nouvelle_tranche_age"] + [f"{a}" for a in range(2012, 2026)]

# 1. Sélection des années voulues (une sur deux, de 2013 à 2025)
annees = list(range(2013, 2026, 2))
cols = [f"{a}" for a in annees]

# 2. Mise en forme : tranches d’âge en colonnes, années en lignes
df_plot = df_medecins_nouvelles_tranches.set_index("Nouvelle_tranche_age")[cols].T

# 3. Passage en proportions (chaque année = 100 %)
#df_plot = df_plot.div(df_plot.sum(axis=1), axis=0)

# 4. Graphique en barres empilées
df_plot.plot(kind="bar",stacked=True,figsize=(10, 6))

plt.ylabel("Proportion")
plt.xlabel("Année")
plt.title("Répartition des médecins (Médecine Générale) par tranche d’âge\n(en proportion, une année sur deux)")
plt.legend(title="Tranche d’âge", bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()
